<img src="https://datasciencecampus.ons.gov.uk/wp-content/uploads/sites/10/2017/03/data-science-campus-logo-new.svg"
             alt="ONS Data Science Campus Logo"
             width = "240"
             style="margin: 0px 60px"
             />

In [32]:
# import the helper functions from the parent directory,
# these help with things like graph plotting and notebook layout
import sys
sys.path.append('..')
from helper_functions import *

# set things like fonts etc - comes from helper_functions
set_notebook_preferences()

# add a show/hide code button - also from helper_functions
toggle_code(title = "import functions")

In [23]:
#dependencies
import pandas as pd
titanic = pd.read_csv('../Data/titanic.csv')

#columns from NewVar script
titanic['child'] = (titanic['age'] < 18).astype(int)
titanic['embarked_city'] = titanic['embarked'].map({'S':'Southampton','C':'Cherbourg','Q':'Queenstown'})
titanic['surname'] = titanic['name'].str.split(',',expand=True)[0]

#columns from descstat script
titanic['z_score'] = (titanic['fare'] - titanic['fare'].mean())/titanic['fare'].std()


toggle_code()

# Merging Data

Often all the data you need to answer a question are not contained within a single dataset, but across several. Datasets can be joined, or 'merged', to allow data to be analysed together, but only **if the two datasets share a common reference or identifier.**

Linking data come in a number of forms, and are commonly referred to as 'indexical' data. Some examples include:

* Your NHS number, allowing data linkage across the NHS for primary, secondary, tertiary care episodes and prescribing.
* Any account number (e.g. banking, utilities, travel card, council tax etc.) can acts as a point of linkage between different sets of data.
* Your email, phone number, social media handles etc.
* Your address can also act as a spatial reference, linking you to your neighbourhood, local services etc.

## 11.1 Different ways to merge

While you may have heard it called "Join" in other languages, particularly in database query languages, in pandas we use the `.merge()` function.

Once you have established that two dataframes share a reference that will permit a merge to be executed, you may wish to further specify how the merge behaves with the `how` parameter.

* Inner - Only rows with reference values that appear in both dataframes are merged.
* Left - All the data from the 'left' dataframe is retained, and any rows that have matching references are merged from the 'right' dataframe.
* Right - All data from the right and anything that matches from the left. Effectively, the reverse of 'Left'.
* Outer (Full) - all data from the left and right dataframe is retained, matched up where possible.

This can be easier to understand graphically:

![joins](https://www.dofactory.com/Images/sql-joins.png)

Let’s read in some additional titanic data and have a look at them.

The new dataset includes the passenger name and age, as well as the additional variables:
* boat - lifeboat identifier
* body - body identification number
* home.dest - the passenger's home and destination in the form "home / dest" or just "home".

The dataset is located in the 'Data' folder, it is an excel file called: 'titanic_more.xlsx'.

In [24]:
# Read in the titanic_more.xlsx using pandas.
titanic_more = pd.read_excel('../Data/titanic_more.xlsx')
titanic_more.head()

,name,age,boat,body,home.dest
0,"Allen, Miss. Elisabeth Walton",29.0000,2,NaN,"St Louis, MO"
1,"Allison, Master. Hudson Trevor",0.9167,11,NaN,"Montreal, PQ / Chesterville, ON"
2,"Allison, Miss. Helen Loraine",2.0000,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,"Allison, Mr. Hudson Joshua Creighton",30.0000,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",25.0000,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In order to merge the two tables we need to use a column which uniquely defines each passenger and is available in both dataframes. At first glance, 'name' would appear to be a good candidate for this, however, remember there are a couple of passengers who have the same name as each other.

**If a common code is available, then it would be advisable to merge on that in preference to any entity name, as codes tend to have a standardised format.**

We can explicitly check if a column uniquely identifies rows with the `Series.is_unique` property.

In [25]:
# Name is not unique defined for each row in titanic
titanic['name'].is_unique

False

In [26]:
# Name is not unique defined for each row in titanic_more
titanic_more['name'].is_unique

False

In this case, we can create a field that will uniquely identify passengers in both datesets by generating a new variable that combines the 'name' and 'age' variables. This is because we happen to know the ages of the passengers who have the same name.

In [27]:
# Create unique id base don name and age for titanic
titanic['name_age_id'] = titanic['name'] + " " + titanic['age'].astype(str)
# Check if the new variable is unique
titanic['name_age_id'].is_unique

True

In [28]:
# Create unique id base don name and age for titanic
titanic_more['nameageid'] = titanic_more['name'] + " " + titanic_more['age'].astype(str)
# Check if the new variable is unique
titanic_more['nameageid'].is_unique

True

## 11.2 Merge function


Now that we have unique id fields in both titanic and titanic_more, we use them to merge the two datasets.

In [29]:
# merge the titanic_more dataset with titanic
titanic_merge = titanic.merge(titanic_more[['name_age_id','boat','body','home.dest']], on = 'name_age_id')
titanic_merge.head()

KeyError: "['name_age_id'] not in index"

In the code cell above we merge the `titanic_more` dataframe into `titanic`. We do this on the basis of the 'name_age_id' variable that we created.

The default merge behaviour is 'inner' join, however in this particular case all behaviours ('inner', 'left', 'right','outer') resolve to the same outcome as both datasets include the same 1,309 passengers.

In [30]:
# Note that pandas actually handles multiple columns directly for unique identification.
titanic_merge = titanic.merge(titanic_more[['name','age','boat','body','home.dest']], on = ['name','age'])

# Exercise 7

Load the revised dataset 'titanic_revised.xlsx' which only includes passengers which have a value for boat, body or home.dest and try merging this to titanic.

1. Which types of merge perform as expected?
    * Try the different `how` parameters: 'inner', 'outer', 'left', 'right'.
2. How many values are observed for boat, body, and home.dest?
    * i.e. How many values are non-missing/
    * Hint: use `.count()`
3. How many passengers in the dataset used lifeboat number 3? What proportion of them were female?
    * Hint: think about datatypes.
4. How many passengers record 'New York' or 'NY' somewhere in the 'home.dest' column?
    * If you do a selection using `Series.str.contains()` you need to specify the parameter na=False.
    * This sets missing values to `False` in the Boolean filter and excludes them from the selection.

In [31]:
## read in titanic-revised dataset
#titanic_revised = pd.read_excel('../Data/titanic_revised.xlsx')

## Create unique id field
#titanic_revised['name_age_id'] = titanic_revised['name'] + " " + titanic_revised['age'].astype(str)

## Merge the titanic datasets

##Question 1

## left is the obvious choice for this join
#titanic_merge2 = titanic.merge(titanic_revised[['name_age_id','boat','body','home.dest']], how = 'left', on = 'name_age_id')

## outer join works too
##titanic_merge2 = titanic.merge(titanic_revised[['name_age_id','boat','body','home.dest']], how = 'outer', on = 'name_age_id')
#print('Left and Outer joins work for merging these datasets.\n')

## Question 2
#print("counts of observations in 'boat', 'body', and 'home.dest'")
#print(titanic_merge2[['boat','body','home.dest']].count(),'\n')

## Question 3

#print("{} passengers used lifeboat 3.".format(len(titanic_merge2[titanic_merge2['boat'] == '3'])))
#print("The proportion of female passengers in lifeboat 3 was {}.\n".format(titanic_merge2[titanic_merge2['boat'] == '3']['sex'].value_counts(normalize=True)['female']))

## Question 4
#NY_count = (titanic_merge2['home.dest'].str.contains('NY') | titanic_merge2['home.dest'].str.contains('New York')).sum()
#print("{} passengers are listed as having either home or destination as 'New York' or 'NY' in the data".format(NY_count))

toggle_code()